# SageMaker Pipeline

### Define Libraries

In [6]:
import sagemaker
from sagemaker.estimator import Estimator as xgb_Estimator
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterString, ParameterInteger, ParameterFloat
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.functions import Join
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.fail_step import FailStep

from sagemaker.sklearn.processing import SKLearnProcessor

import os

### Define variables

In [7]:
sagemaker_role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
pipeline_session = PipelineSession()

aws_region = sagemaker_session.boto_session.region_name

image_uri = image_uris.retrieve(
    framework="xgboost",          # Framework name
    region=aws_region,            # Region (defaults to the region of the session)
    version="1.5-1",              # Container version
    #instance_type="ml.m5.large",  # Instance type (optional, used for multi-model endpoints)
    py_version="py3"              # Python version (optional, default is "py3")
)

# s3_bucket = sagemaker_session.default_bucket()
s3_bucket = "aamlops2024"
s3_prefix = "xgb-pipeline-lam-01"
s3_base_url = f"s3://{s3_bucket}/{s3_prefix}"
s3_bucket, s3_prefix, s3_base_url, aws_region, image_uri

[11/27/24 19:51:52] INFO     Ignoring unnecessary Python version: py3.                            ]8;id=596619;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=660063;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#596\596]8;;\

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=516608;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=609709;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#523\523]8;;\

('aamlops2024',
 'xgb-pipeline-lam-01',
 's3://aamlops2024/xgb-pipeline-lam-01',
 'us-east-1',
 '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1')

### Define the pipeline parameters

In [8]:
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.m5.xlarge")
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
batch_instance_type = ParameterString(name="BatchInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="PendingManualApproval")
input_data_uri = ParameterString(name="InputDataUri", default_value=f"s3://{s3_bucket}/{s3_prefix}/data/data.csv")
output_path = ParameterString(name="OutputPath", default_value=f"s3://{s3_bucket}/{s3_prefix}/output")
accuracy_threshold = ParameterFloat(name="AccuracyThreshold", default_value=0.85)
model_package_group_name="xgb-Model-01"

### Step 0: Get Data

In [9]:
import boto3
s3 = boto3.client("s3")
s3.download_file(
    f"sagemaker-example-files-prod-{sagemaker_session.boto_region_name}",
    "datasets/tabular/synthetic/churn.txt",
    "data/churn.txt",
)

In [10]:
import pandas as pd
data = pd.read_csv("./data/churn.txt")

In [11]:
data.to_csv("./data/data.csv")

In [12]:
# Mode data to S3
local_path = "./data/data.csv"
destination_path = os.path.join(s3_base_url, "data")
s3_input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=destination_path
)
s3_input_data_uri

's3://aamlops2024/xgb-pipeline-lam-01/data/data.csv'

In [13]:
input_data_uri

ParameterString(name='InputDataUri', parameter_type=<ParameterTypeEnum.STRING: 'String'>, default_value='s3://aamlops2024/xgb-pipeline-lam-01/data/data.csv')

In [14]:
# Define cache configuration
cache_config = CacheConfig(
    enable_caching=True,             # Enable caching
    expire_after="P30D"              # Cache expiry in ISO 8601 duration format (e.g., P30D = 30 days)
)

### Step 2: Data Preprocessing

In [15]:
sklearn_processor = SKLearnProcessor(
    framework_version="1.2-1",
    instance_type=processing_instance_type,
    instance_count=1,
    role=sagemaker_role,
    sagemaker_session=pipeline_session,
    base_job_name="xgb_pipeline",
)

[11/27/24 19:51:53] WARNING  The input argument instance_type of function                          ]8;id=87155;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=199173;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.retrieve) is a pipeline variable (<class                        
                             'sagemaker.workflow.parameters.ParameterString'>), which is                           
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    INFO     Defaulting to only available Python version: py3                     ]8;id=990375;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=217541;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#603\603]8;;\

In [16]:
# procesor_args = sklearn_processor.run (
#     inputs=[
#         ProcessingInput(source=input_data_uri, destination="/opt/ml/processing/input")
#     ],
#     outputs=[
#         ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
#         ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
#         ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
#     ],
#     code="./code/preprocess.py",  # your custom preprocessing script
# )

# step_process = ProcessingStep (name="PreprocessData", step_args=procesor_args, cache_config=cache_config)

In [17]:
# !pip install -U sagemaker

In [18]:
image_uri_pyspark = sagemaker.image_uris.retrieve(
        framework="spark",
        region=aws_region,
        version="2.4",
        py_version="py37")
image_uri_pyspark

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=204559;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=474822;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#527\527]8;;\

'173754725891.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark-processing:2.4-cpu-py37'

In [19]:
spark_processor = ScriptProcessor(
    image_uri=image_uri_pyspark,
    command=["/opt/program/submit"],
    role=sagemaker_role,
    instance_type=processing_instance_type,
    instance_count=1,
    sagemaker_session=pipeline_session,
    base_job_name= "Xgb_PySpark",
)

In [20]:
procesor_args = spark_processor.run(
    inputs=[
        ProcessingInput(source=input_data_uri, destination="/opt/ml/processing/input")
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
    ],
    code="./code/preprocesspyspark.py",
    arguments=["--input_data", input_data_uri, "--output", input_data_uri],
)

step_process = ProcessingStep(name="PreprocessData", step_args=procesor_args, cache_config=cache_config)

# step_process = ProcessingStep(
#     name="PreprocessData",
#     processor=script_processor_lineage,
#     inputs=[
#         ProcessingInput(source=input_data_uri, destination="/opt/ml/processing/input")
#     ],
#     outputs=[
#         ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
#         ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
#         ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=f"s3://{s3_bucket}/{s3_prefix}/processing"),
#     ],
#     code="./code/preprocesspyspark.py",
#     cache_config=cache_config
# )

/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


### Step 2: Training

In [21]:
xgb_estimator = xgb_Estimator(
    image_uri=sagemaker.image_uris.retrieve(framework="xgboost", region=pipeline_session.boto_session.region_name, version="1.5-1"),
    entry_point="code/train.py",  # your custom training script
    instance_type=training_instance_type,
    instance_count=1,
    role=sagemaker_role,
    output_path=output_path,
    sagemaker_session=pipeline_session,
    base_job_name="TrainXGBoostModel",
    hyperparameters={
        "objective": "binary:logistic",
        # "num_round": 100,
        "num_round": 5,
        "max_depth": 5,
        "eta": 0.2,
        "gamma": 4,
        "min_child_weight": 6,
        "subsample": 0.8
    }
)

train_args = xgb_estimator.fit(
    inputs={
        "train": TrainingInput(step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri, content_type="text/csv"),
        "validation": TrainingInput(step_process.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri, content_type="text/csv"),
    }
)

step_train = TrainingStep (name="TrainXGBoostModel", step_args=train_args, cache_config=cache_config)

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=89647;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=954204;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#523\523]8;;\

### Step 3: Model Evaluation

In [22]:
script_processor_eval = ScriptProcessor(
    image_uri=image_uri,  # Correct URI for us-east-1
    command=["python3"],
    role=sagemaker_role,
    instance_type=processing_instance_type,  # Choose the instance type
    instance_count=1,
    sagemaker_session=pipeline_session,
    base_job_name= "Xgb_Evaluation",
)

In [23]:
eval_args = script_processor_eval.run(
    inputs=[
        ProcessingInput(source=step_train.properties.ModelArtifacts.S3ModelArtifacts, destination="/opt/ml/processing/model" ),
        ProcessingInput(source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri, destination="/opt/ml/processing/test"),
    ],
    outputs=[ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=f"s3://{s3_bucket}/{s3_prefix}/evaluation")],
    code="code/evaluate.py"  # your custom evaluation script
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json",
)

step_eval = ProcessingStep(name="EvaluateModel", step_args=eval_args, property_files=[evaluation_report], cache_config=cache_config )

### Model Registey Step

In [24]:
from sagemaker.model_metrics import ModelMetrics
from sagemaker.model_metrics import MetricsSource

model_metrics = ModelMetrics(
  model_statistics=MetricsSource(
    s3_uri="{}/evaluation.json".format(
      step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    ),
    content_type="application/json"
  )
)

[11/27/24 19:51:54] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=964704;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=472319;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

In [25]:
step_register = RegisterModel(
    name="MyModel",
    estimator=xgb_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.t2.medium"],    
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    depends_on=[step_eval]
)

### Model Lineage

In [26]:
script_processor_lineage = ScriptProcessor(
    image_uri=image_uri, 
    command=["python3"],
    role=sagemaker_role,
    instance_type=processing_instance_type,
    instance_count=1,
    sagemaker_session=pipeline_session,
    base_job_name= "Xgb_Lineage",
)

In [27]:
lineage_args = script_processor_lineage.run(
    inputs=[
        ProcessingInput(
            source=step_eval.properties.ProcessingOutputConfig.Outputs["evaluation"].S3Output.S3Uri,
            destination="/opt/ml/processing/evaluation",
        ),
    ],
    code="code/lineage.py",
    arguments=[
        "--evaluation-uri",
        step_eval.properties.ProcessingOutputConfig.Outputs["evaluation"].S3Output.S3Uri,
        "--model-package-arn",
        step_register.properties.ModelPackageArn,
        "--region",
        "us-east-1",  # Pass your desired AWS region here
        "--model-artifact-url",
        step_train.properties.ModelArtifacts.S3ModelArtifacts,
    ],
)

step_lineage = ProcessingStep(name="EvaluationLineage", step_args=lineage_args)

### Condition Step

In [28]:
step_fail = FailStep(
    name="AccuracyFailed",
    error_message=Join(on=" ", values=["Execution failed due to model accuracy < ", accuracy_threshold ]),
)

In [29]:
cond_lte = ConditionGreaterThan(  # You can change the condition here
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.accuracy.value"  # This should follow the structure of your report_dict defined in the evaluate.py file.
    ),
    right=accuracy_threshold,  # You can change the threshold here
)

In [30]:
step_cond = ConditionStep(
    name="Check-Accuracy",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[step_fail]
)

### Define Pipeline Parameters, Steps and Conditions

In [31]:
pipeline = Pipeline(
    name="xgb-pipeline",
    sagemaker_session=sagemaker_session,
    parameters=[
                processing_instance_type,
                instance_type, 
                model_approval_status,
                input_data_uri,
                output_path,
                accuracy_threshold,
               ],
    steps=[step_process, step_train, step_eval, step_cond, step_lineage]
)

In [32]:
pipeline.upsert(role_arn=sagemaker_role)

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=217082;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=874321;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=430804;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=109624;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=223065;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=514538;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  No finished training job found associated with this estimator.       ]8;id=254292;file:///opt/conda/lib/python3.11/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=527487;file:///opt/conda/lib/python3.11/site-packages/sagemaker/estimator.py#1909\1909]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  Popping out 'CertifyForMarketplace' from the pipeline definition since   ]8;id=548681;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/_utils.py\_utils.py]8;;\:]8;id=881913;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/_utils.py#515\515]8;;\
                             it will be overridden in pipeline execution time.                                     

...

                    WARNING  No finished training job found associated with this estimator.       ]8;id=49249;file:///opt/conda/lib/python3.11/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=624121;file:///opt/conda/lib/python3.11/site-packages/sagemaker/estimator.py#1909\1909]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  Popping out 'CertifyForMarketplace' from the pipeline definition since   ]8;id=821756;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/_utils.py\_utils.py]8;;\:]8;id=579058;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/_utils.py#515\515]8;;\
                             it will be overridden in pipeline execution time.                                     

                    WARNING  Popping out 'ModelPackageName' from the pipeline definition by        ]8;id=535916;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=625878;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=625202;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=241513;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:047922237497:pipeline/xgb-pipeline',
 'ResponseMetadata': {'RequestId': '17e49ff9-6d37-4953-b503-ef591de3324b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '17e49ff9-6d37-4953-b503-ef591de3324b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '80',
   'date': 'Wed, 27 Nov 2024 19:51:54 GMT'},
  'RetryAttempts': 0}}

In [33]:
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:047922237497:pipeline/xgb-pipeline/execution/lxgx9ohsz5fl', sagemaker_session=<sagemaker.session.Session object at 0x7f5c7f588550>)

In [34]:
!aws s3 cp s3://aamlops2024/xgb-pipeline-lam-01/evaluation/evaluation.json .

download: s3://aamlops2024/xgb-pipeline-lam-01/evaluation/evaluation.json to ./evaluation.json


In [35]:
from sagemaker import image_uris

image_uri_pyspark = image_uris.retrieve(
    framework="spark",
    region="us-east-1",  # Replace with your region
    version="2.4",
    py_version="py37",
    instance_type="ml.m5.xlarge",  # Adjust instance type as needed
)
print(image_uri_pyspark)

173754725891.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark-processing:2.4-cpu-py37
